<a href="https://colab.research.google.com/github/minstradamuss/Fishing-Classification/blob/main/LLaVA_7b_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install ninja
!pip install flash-attn --no-build-isolation

!pip install -e .

# !python -m llava.serve.cli \
#     --model-path liuhaotian/llava-v1.5-7b \
#     --image-file "https://llava-vl.github.io/static/images/view.jpg"

/content
Cloning into 'LLaVA'...
remote: Enumerating objects: 1960, done.
remote: Total 1960 (delta 0), reused 0 (delta 0), pack-reused 1960 (from 1)
Receiving objects: 100% (1960/1960), 13.60 MiB | 31.02 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.
/content/LLaVA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [ ]:
import threading
import subprocess
threading.Thread(target=lambda: subprocess.run(['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'], check=True), daemon=True).start()

In [ ]:
import threading
import subprocess
command = [
    'python3', '-m', 'llava.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', 'liuhaotian/llava-v1.5-7b'
]
threading.Thread(target=lambda: subprocess.run(command, check=True, shell=False), daemon=True).start()

In [4]:
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share

/usr/bin/python3: Error while finding module specification for 'llava.serve.gradio_web_server' (ModuleNotFoundError: No module named 'llava')


In [5]:
import requests

image_path = "/content/1.jpg"

prompt = """На изображении скриншот сайта.
Если там форма ввода аутентификационных данных, то кластер сайта "логинка". Платформа, куда производится вход.
Если там есть формы голосования, то кластер сайта считаем "голосование". У "голосования" нет платформы.
Если там предлагаются инвестиции, то кластер сайта считаем "инвестиции". Укажи платформу, в которую вкладываются деньги.
Если производится розыгрыш лотерей, то кластер сайта - "лотерея". Платформа - компания проводящая лотерею.
Если не находится кластер из выше написанного, то кластер сайта - "чистый".
Дай краткий ответ в виде: кластер сайта, платформа"""

url = "http://localhost:7860/run/predict"  # Порт может отличаться, если ты указал другой при запуске

# Чтение изображения
with open(image_path, "rb") as img_file:
    files = {"image": img_file}
    data = {
        "data": [prompt]
    }
    response = requests.post(url, files=files, data=data)

print(response.json())


ConnectionError: HTTPConnectionPool(host='localhost', port=7860): Max retries exceeded with url: /run/predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f813425dcd0>: Failed to establish a new connection: [Errno 111] Connection refused'))